7/2/18:
this is ment to be a temporary notebook where I will experiment with how to run the mimic SOFA score sql code. The code requires having a number of materialized views open, and this is somthing i am unfamiliar with how to handle in jupyter. The purpose of the notebook will be to learn how to utilize these materialized views to run the sofa score script.

upon completion of this, i intend on deleting this file and incorporating the lessons I learn into my labnotebook and my main 03-clinical_variables_streamlined.ipnb notebook

In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
import urllib3
import prettytable
from collections import Counter
import seaborn as sns
import random

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
import psycopg2
conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

time: 2.05 s


## cohort

In [3]:
#patients of interest from rotation_cohort_generation

final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/final_pt_df2_5-04-18.csv', index_col=0)

patients= list(final_pt_df2['subject_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 57.7 ms


## functions

In [4]:
def sql_exe_show(sql_sentence):
    cur.execute(sql_sentence)
    rows = cur.fetchall()
    col = []
    for i in range(len(cur.description)):
        col.append(cur.description[i][0])
    table = pd.DataFrame(rows,columns=col)
    return table

#input the sql_exe_show object and get dataframe for only patients in patient list out. 
def sql_to_df_icu(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['icustay_id'].isin(icustay_id)]
    return sql_exe_show_df

def sql_to_df_patients(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['subject_id'].isin(patients)]
    return sql_exe_show_df

time: 13.8 ms


from the github:
    
How to use code in this folder
This folder generates materialized views for a variety of severity of illness scores (detailed below). The queries make use of materialized views which aggregate data from the first day of a patient's ICU stay. To run the code, it is necessary to:

Run the scripts which generate the constituent materialized views
echodata - generated by /concepts/echo-data.sql
ventdurations - generated by /concepts/durations/ventilation-durations.sql - (needed for subsequent view)
vitalsfirstday - generated by /concepts/firstday/vitals-first-day.sql
uofirstday - generated by /concepts/firstday/urine-output-first-day.sql
ventfirstday - generated by /concepts/firstday/ventilated-first-day.sql
gcsfirstday - generated by /concepts/firstday/gcs-first-day.sql
labsfirstday - generated by /concepts/firstday/labs-first-day.sql
bloodgasfirstday - generated by /concepts/firstday/blood-gas-first-day.sql - (needed for subsequent view)
bloodgasfirstdayarterial - generated by /concepts/firstday/blood-gas-first-day-arterial.sql


Run the script for the severity of illness score you are interested in
OASIS - oasis.sql
SAPS - saps.sql
SOFA - sofa.sql

file location of mimic first day codes (just add on the ending posted above next to each script):
/Users/geickelb1/Documents/GitHub/mimic-code


In [ ]:
#example of how i've been running my materialized view scripts. 

f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/ventparam_all48.sql', 'r') 
ventparamdata = f.read()
ventparam_all48_sql = ventparamdata.format(tuple(icustay_id))  
cur.execute('rollback')
cur.execute(ventparam_all48_sql)
ventparamTable= sql_exe_show('select * from ventparam_all48;')
ventparam_all48_df= sql_to_df_icu(ventparamTable)